In [4]:
import pandas as pd
import gzip
from funciones import extraer_anio
from funciones import convertir_0

In [5]:
games = pd.read_csv(r'data_sets\steam_gamez.csv',index_col=False)

C:\Users\jonat\AppData\Local\Temp\ipykernel_11480\2807454174.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv(r'data_sets\steam_gamez.csv',index_col=False)


In [6]:
# Verifico que las filas que tengo pensado eliminar, solo esten rellenas con valores NaN, de lo contrario, en pasos siguientes verificare como realizar la normalizacion
games[games.isna().all(axis=1)].shape[0] + games.dropna(thresh=1).shape[0]

120445

In [7]:
# Elimino las filas nulas


games.dropna(thresh=9,inplace=True) # ---> Elimino las filas que cumplen con la condicion de estar completamente sin datos (NaN), de lo contrario si contienen algun valor no se eliminan


games.drop(columns='title',inplace=True) # ---> Elimino la columna title ya que la columna app_name contien el mismo valor y con los faltantes


games = games.drop(games.loc[games['id'].isna()].index) # ---> Realice una exploracion en la columna id, y tome la desicion de eliminar las filas que no contienen valores, no tienen relevancia en el resto del proceso


games['id'] = games['id'].astype(int) # ---> Realice una conversion en el tipo de dato de la columna id, de Float a Int


games = games.drop_duplicates(subset=['id']) # ---> Comprobe que existen valores repetidos, verifique que todos sus valores sean identicos y luego procedi a realizar su eliminacion.


# Primero comprobe que la columna genres y tags, contienen los mismos valores, complete los faltantes de la columna tags con los valores de la columna genres
filas_vacias = games['tags'].isna() # ---> guardo las filas con valores NaN en tags
games.loc[filas_vacias, 'tags'] = games.loc[filas_vacias, 'genres'] # asigno los valores de la columna genres dentro de la columna tags
games.drop(columns='genres',inplace=True) # ---> Elimino la columna genres


# Relleno los valores de la columna publisher, con los valores de la columna developer, solo cuando publisher no tiene ningun valor.
filas_vacias = games['publisher'].isna()
games.loc[filas_vacias, 'publisher'] = games.loc[filas_vacias, 'developer']
# En este caso no elimino ninguna de las columnas, ya que algunos valores que son relevantes tambien son diferentes.


games.dropna(inplace=True) # ---> Elimino todos los NaN

games['release_date'] = games['release_date'].apply(extraer_anio)

games['release_date'] = games['release_date'].replace('SOON™',2019)
games['release_date'] = games['release_date'].replace('soon',2019)
games['release_date'] = games['release_date'].replace('SOON',2019)

games['price'] = games['price'].apply(convertir_0)


games['id'] = games['id'].astype(int)


games['release_date'] = games['release_date'].astype(int)

games.shape

(27220, 11)

In [10]:
games.head()

,publisher,app_name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140,Kotoshiro
88311,"Making Fun, Inc.",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",0.00,0.0,643980,Secret Level SRL
88312,Poolians.com,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",0.00,0.0,670290,Poolians.com
88313,彼岸领域,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,0.0,767400,彼岸领域
88315,Trickjump Games Ltd,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018,"['Action', 'Adventure', 'Simulation', 'FPS', '...",http://steamcommunity.com/app/772540/reviews/?...,"['Single-player', 'Steam Achievements']",3.99,0.0,772540,Trickjump Games Ltd


In [11]:
games.isna().sum()

publisher       0
app_name        0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

In [12]:
games.to_csv('data_set_limpio/games_preparado.csv',encoding='UTF-8',index=False)

In [13]:
with open('data_set_limpio/games_preparado.csv', 'rb') as f_in, gzip.open('data_set_limpio/games_preparado.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)